In [1]:
import numpy as np
import cupy as cp
import pandas as pd
import networkx as nx
from numba import cuda
import time
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("CA-GrQc.csv")

In [3]:
df.head()

,FromNodeId,ToNodeId
0,3466,937
1,3466,5233
2,3466,8579
3,3466,10310
4,3466,15931


In [4]:
df.shape[0] # number of edges

28980

In [5]:
# making the graph
G = nx.from_pandas_edgelist(df, source='FromNodeId', target='ToNodeId')

In [6]:
# draw the graph

plt.figure(figsize=(100, 50))
nx.draw(
    G,
    with_labels=False,
    node_color='red',
    edge_color='black',
    node_size=1000,
    font_size=8
)
plt.title("Graph of LastFM User Network")
plt.show()


In [7]:
def print_matrix(matrix):
    for row in matrix:
        print(" ".join(f"{val:.2f}" for val in row))

In [8]:
adj_lastfm = nx.to_numpy_array(G, dtype=np.float32)

print("Numpy Adjacency Matrix:\n", adj_lastfm)

Numpy Adjacency Matrix:
 [[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]


In [9]:
def Normalize(matrix):
    matrix2 = matrix.copy()

    for j in range(len(matrix2[0])):
        temp = 0
        for i in range(len(matrix2)):
            temp += matrix2[i][j]
        for i in range(len(matrix2)):
            matrix2[i][j] = matrix2[i][j] / temp
        
    return matrix2

def Inflate(matrix, r):
    matrix = cp.power(matrix, r)
    col_sums = cp.sum(matrix, axis=0)
    matrix = matrix / col_sums
    return matrix

@cuda.jit
def matmul(A, B, C):
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.0
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

def powerMatrix(matrix, n, bpg, tpb):
    if n < 1:
        raise ValueError("Pangkat matrix harus >= 1")

    matrix1 = matrix
    matrix2 = cp.zeros_like(matrix)

    for i in range(n - 1):
        matmul[bpg, tpb](matrix1, matrix, matrix2)
        cuda.synchronize()  # penting untuk sinkronisasi kernel
        matrix1 = matrix2.copy()

    return matrix1

def print_matrix(matrix):
    for row in matrix:
        print(" ".join(f"{val:.2f}" for val in row))

def MarkovCluster(matrix_a, e, r, iter):
    matrix = matrix_a

    # Add self-loops
    for i in range(len(matrix)):
        matrix[i][i] = 1

    matrix = Normalize(matrix)

    print("Normalized Matrix:\n", matrix)

    matrix = cp.asarray(matrix)

    # Setup CUDA kernel execution config
    threadsperblock = (16, 16)
    blockspergrid_x = int(np.ceil(matrix.shape[0] / threadsperblock[0]))
    blockspergrid_y = int(np.ceil(matrix.shape[1] / threadsperblock[1]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    iterate = iter
    tol = 1e-7
    ptr = 0
    converged = False

    while ptr < iterate and not converged:
        prev_matrix = matrix.copy()

        matrix = powerMatrix(matrix, e, bpg=blockspergrid, tpb=threadsperblock)
        matrix = Inflate(matrix, r)

        # Cek konvergensi
        diff = cp.abs(matrix - prev_matrix)
        max_diff = cp.max(diff)
        if max_diff < tol:
            converged = True

        ptr += 1

    matrix = matrix.get()
    return matrix

In [10]:
e_1 = 2
r_1 = 2
start = time.time()
MarkovMatrix = MarkovCluster(matrix_a=adj_lastfm, e = e_1, r = r_1, iter = 20)
end = time.time()

Normalized Matrix:
 [[0.11111111 0.16666667 0.33333334 ... 0.         0.         0.        ]
 [0.11111111 0.16666667 0.         ... 0.         0.         0.        ]
 [0.11111111 0.         0.33333334 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.33333334 0.33333334 0.33333334]
 [0.         0.         0.         ... 0.33333334 0.33333334 0.33333334]
 [0.         0.         0.         ... 0.33333334 0.33333334 0.33333334]]


In [11]:
print(MarkovMatrix)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.33333334 0.33333334 0.33333334]
 [0.         0.         0.         ... 0.33333334 0.33333334 0.33333334]
 [0.         0.         0.         ... 0.33333334 0.33333334 0.33333334]]


In [12]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# === 1. MCL Matrix Sample ===
matriks_mcl = MarkovMatrix[:1000, :1000]

# === 2. Extract Clusters ===
def extract_clusters(mtx, threshold=0.01):
    binary = (mtx > threshold).astype(int)
    clusters = []
    visited = set()
    for i in range(binary.shape[0]):
        if i in visited:
            continue
        cluster = set(np.nonzero(binary[i])[0])
        if cluster:
            clusters.append(tuple(cluster))
            visited.update(cluster)
    return clusters

clusters = extract_clusters(matriks_mcl)

# === 3. Create Graph ===
adj_matrix = (matriks_mcl > 0.01).astype(int)
G = nx.from_numpy_array(adj_matrix)

# Assign cluster IDs
cluster_dict = {}
for cluster_id, cluster in enumerate(clusters):
    for node in cluster:
        cluster_dict[node] = cluster_id

# === 4. Draw and Save PNG ===
pos = nx.spring_layout(G, seed=42)
node_colors = [cluster_dict.get(node, -1) for node in G.nodes]

plt.figure(figsize=(30, 20))
nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.5)
nx.draw_networkx_nodes(
    G, pos,
    node_color=node_colors,
    cmap=plt.cm.Set3,
    node_size=250,
    edgecolors='gray'
)
# nx.draw_networkx_labels(G, pos)

plt.title("Visualisasi Cluster dari MCL")
plt.axis('off')
plt.tight_layout()

# ✅ Save to PNG
plt.savefig("mcl_graph.png", dpi=300)
plt.close()

print("✅ Gambar disimpan sebagai mcl_graph.png")


✅ Gambar disimpan sebagai mcl_graph.png


In [13]:
from PIL import Image
Image.open("mcl_graph.png").show()


In [14]:
def get_clusters_from_square_matrix(mcl_matrix):
    """
    mcl_matrix: square numpy array (n_nodes x n_nodes), hasil dari MCL
                dengan 1s menunjukkan koneksi antar node dalam cluster
    return: list of clusters, each cluster is a list of node indices
    """
    # Buat graph dari adjacency matrix
    G = nx.from_numpy_array(mcl_matrix)

    # Ambil connected components (cluster)
    clusters = [list(component) for component in nx.connected_components(G)]

    return clusters

clusters = get_clusters_from_square_matrix(MarkovMatrix)

for i, cluster in enumerate(clusters):
    print(f"Cluster {i+1}: {cluster}")

Cluster 1: [0, 3, 5, 7, 8]
Cluster 2: [1, 740, 741, 742, 743, 746, 747, 2218, 2219, 750, 751, 2220, 2221, 2222, 755, 756, 504]
Cluster 3: [2, 18, 4, 14]
Cluster 4: [2787, 1797, 1798, 6, 1790, 1791]
Cluster 5: [9, 3730, 3731, 3732]
Cluster 6: [1649, 10, 4634]
Cluster 7: [576, 387, 579, 580, 583, 584, 585, 11, 139, 1180, 570, 572, 574, 575]
Cluster 8: [12]
Cluster 9: [13]
Cluster 10: [932, 936, 1614, 1615, 15, 1616, 1617, 3347]
Cluster 11: [68, 69, 4164, 71, 4165, 4166, 75, 79, 16, 86, 87, 89, 90]
Cluster 12: [2144, 4416, 2145, 4417, 17, 4415]
Cluster 13: [3553, 3554, 3555, 3556, 2949, 3557, 3558, 3559, 3560, 3561, 3562, 3563, 3564, 3565, 3567, 3568, 19]
Cluster 14: [836, 38, 39, 46, 20, 22, 26, 28, 30]
Cluster 15: [232, 629, 21, 826, 827, 828, 829]
Cluster 16: [3040, 3041, 1192, 3409, 1982, 23, 1981, 3038, 3039]
Cluster 17: [24, 698, 3195, 3196]
Cluster 18: [3552, 2612, 25, 3547, 3548, 3549, 3550, 3551]
Cluster 19: [35, 51, 52, 56, 57, 58, 27, 61]
Cluster 20: [2333, 2329, 2330, 2331, 23